# Nyan Cafe Simulation  
Esta es una simulación de una cafetería que consta de ```<nTable>``` mesas, cada una con una capacidad de ```<tableCapacity>``` personas.
Cada ```<spawnTime>``` minutos se generá un grupo de personas de tamaño ```<groupSize>``` que vienen juntas a ocupar una mesa.  
  
Cuando un grupo de personas llega a la cafetería, solicitan una mesa (aleatoriamente); si la mesa no esta disponible, quedan en cola, esperando a que se desocupe la mesa; si ```<groupSize>``` es mayor a ```<tableCapacity>``` el grupo se despacha sin atender; en caso de que la mesa sí este disponible, el grupo pasa a ocuparla.  
  
Cuando el grupo de personas toma una mesa, hacen un pedido de ```<groupSize>``` cafés a un gato robot (seleccionado aleatoriamente), el cúal los atenderá si esta desocupado; si no esta desocupado, los pondrá en cola; en caso de no tener más suministros, despachará a las personas de la mesa y contabilizará como no atendidas.  
  
Cuando el grupo de personas es atendido por el gato robot, este tardará ```<groupSize> X <coffeeTime>``` minutos en preparar sus cafés, luego el gato robot los servirá y pasará a estar desocupado, pero la mesa seguirá ocupada por el grupo de personas por ```<pDrinkingTime>``` minutos más antes de ser desocupada.  
  
Por cada taza de café, los gatos robots gastan ```<gramsPerCoffee>``` gramos de café de su tanque de ```<tankCapacity>``` gramos de capacidad.  
La cafetería tendrá una cantidad de ```<nCatRobot>``` gatos robots funcionando en ella.

## Descripción de variables y enfoque
En este caso se hará enfoque en la **cantidad de mesas vs la cantidad de gatos robot**.  
¿Cómo se comportará la simulación según asignemos mayor o menor cantidad de mesas vs mayor o menor cantidad gatos robot?  
  
Teniendo en cuenta lo anterior, asignaremos ya algunas variables por defecto:

In [261]:
# Importamos las librerias necesarias
import simpy
import pandas as pd
import random

# Definimos algunas variables por defecto
RANDOM_SEED = 42
tableCapacity = 4 # Capacidad de las mesas (Fija a 4 personas)
spawnTime = 40 # Tiempo máximo de llegada de los grupos clientes (1 a 40 minutos)
groupSize = 4 # Tamaño máximo de los grupos de clientes (1 a 4 personas)

pMinDrinkingTime = 10 # Tiempo mínimo de consumo de una taza de café (10 minutos)
pDrinkingTime = 30 # Tiempo máximo de consumo de una taza de café (30 minutos)

coffeeTime = 5 # Tiempo máximo de preparación de una taza de café (1 a 5 minutos)
gramsPerCoffee = 5 # Gasto de café por taza (Fijo a 5 gramos)
tankCapacity = 1000 # Capacidad en gramos del tanque de café de los gatos robots (Fijo a 1000 gramos)

# Definimos los eventos que pueden ocurrir
class Event:
    TOO_MANY_PEOPLE = "TOO_MANY_PEOPLE"
    WAITING_IN_LINE = "WAITING_IN_LINE"
    TABLE_TAKEN = "TABLE TAKEN",
    NOT_ENOUGH_COFFEE = "NOT_ENOUGH_COFFEE"
    MAKING_COFFEE = "MAKING_COFFEE"
    COFFEE_REQ_FINISHED = "COFFEE_REQ_FINISHED"
    DRINKING_COFFEE = "DRINKING_COFFEE"

event = Event()


## Definición de las clases

In [262]:
class Table:
    def __init__(self, env, name, capacity):
        self.env = env
        self.name = name
        self.capacity = capacity # Capacidad de la mesa (En personas) *NO CONFUNDIR CON LA CAPACIDAD DEl RESOURCE*
        self.resource = simpy.Resource(env, capacity=1)

        self.dfT = pd.DataFrame({
            "name": [name],
            "capacity": [capacity]
        })

        self.dfE = pd.DataFrame() # Dataframe para recolectar todos los eventos

    def receiveGroup(self, group):
        print(f"[{self.env.now }] {group.name}: Esta revisando {self.name}")
        yield self.env.timeout(1)
        if self.capacity < group.size:
            print(f"[{self.env.now}] {group.name}: No encontró suficiente espacio en {self.name} (gSize:{group.size}) (tSize:{self.capacity})")
            df = pd.DataFrame({
                "time": [self.env.now],
                "robocat": [group.robocat.name],
                "table": [self.name],
                "group": [group.name],
                "group_size": [group.size],
                "event": [event.TOO_MANY_PEOPLE]
            })
            self.dfE = pd.concat( [ self.dfE , df ] )

        else:
            print(f"[{self.env.now}] {group.name}: Encontró suficiente espacio en {self.name} (gSize:{group.size}) (tSize:{self.capacity})")
            df = pd.DataFrame({
                "time": [self.env.now],
                "robocat": [group.robocat.name],
                "table": [self.name],
                "group": [group.name],
                "group_size": [group.size],
                "event": [event.TABLE_TAKEN]
            })
            self.dfE = pd.concat( [ self.dfE , df ] )
            group.hasTable = True



In [263]:
class Group: # Grupo de clientes
    def __init__(self, env, name, size, drinkingTime, table, robocats):
        self.env = env
        self.name = name
        self.size = size
        self.drinkingTime = drinkingTime
        self.table = table
        self.robocats = robocats
        self.hasTable = False
        self.wasAttended = False
        self.robocat = random.choice(robocats)

        self.dfG = pd.DataFrame({
            "name": [name],
            "size": [size],
            "drinkingTime": [drinkingTime],
            "table": [table.name]
        })

        self.dfE = pd.DataFrame() # Dataframe para recolectar todos los eventos

        env.process(self.in_line())

    def in_line(self): 
        print(f"[{self.env.now}] {self.name}: Esta esperando la {self.table.name}")
        df = pd.DataFrame({
            "time": [self.env.now],
            "robocat": [self.robocat.name],
            "table": [self.table.name],
            "group": [self.name],
            "group_size": [self.size],
            "event": [event.WAITING_IN_LINE]
        })
        self.dfE = pd.concat([self.dfE, df])

        with self.table.resource.request() as table_request:
            yield table_request
            print(f"[{self.env.now}] {self.name}: Ha tomado la {self.table.name}")
            yield self.env.process(self.table.receiveGroup(self))
            if self.hasTable:
                df = pd.DataFrame({
                    "time": [self.env.now],
                    "robocat": [self.robocat.name],
                    "table": [self.table.name],
                    "group": [self.name],
                    "group_size": [self.size],
                    "event": [event.TABLE_TAKEN]
                })
                self.dfE = pd.concat([self.dfE, df])
                print(f"[{self.env.now}] {self.name}: Esperando para pedir a {self.robocat.name}")
                
                with self.robocat.resource.request() as robocat_request:
                    yield robocat_request
                    print(f"[{self.env.now}] {self.robocat.name}: Escuchando a {self.name} ")
                    yield self.env.process(self.robocat.makeCoffee(self, self.table))
                if self.wasAttended:
                    print(f"[{self.env.now}] {self.name}: Recibió su café de {self.robocat.name} y se estará {self.drinkingTime} minutos en la mesa")
                    yield self.env.timeout(self.drinkingTime) # Tiempo de consumo de café
                    df = pd.DataFrame({
                        "time": [self.env.now],
                        "robocat": [self.robocat.name],
                        "table": [self.table.name],
                        "group": [self.name],
                        "group_size": [self.size],
                        "event": [event.DRINKING_COFFEE]
                    })
                self.dfE = pd.concat([self.dfE, df])
                df = pd.DataFrame({
                    "time": [self.env.now],
                    "robocat": [self.robocat.name],
                    "table": [self.table.name],
                    "group": [self.name],
                    "group_size": [self.size],
                    "event": [event.COFFEE_REQ_FINISHED]
                })
                self.dfE = pd.concat([self.dfE, df])
            print(f"[{self.env.now}] {self.name}: Abandonó la mesa {self.table.name}")


In [264]:
class RoboCat: # Gato robot
    def __init__(self, env, name, tankCapacity, coffeeTime):
        self.env = env
        self.name = name
        self.tankCapacity = tankCapacity
        self.coffeeTime = coffeeTime
        self.tank = simpy.Container(env, init=tankCapacity, capacity=tankCapacity)
        self.resource = simpy.Resource(env, capacity=1)

        self.dfR = pd.DataFrame({
            "name": [name],
            "tankCapacity": [tankCapacity],
            "coffeeTime": [coffeeTime]
        })

        self.dfE = pd.DataFrame() # Dataframe para recolectar todos los eventos

    def makeCoffee(self, group, table):

        totalCoffeeGrams = group.size * gramsPerCoffee
        print(f"[{self.env.now}] {self.name}: Procesando pedido de {group.name} en {group.table.name} (pedido:{totalCoffeeGrams}g) (tank:{self.tank.level}/{self.tankCapacity}g)")

        if self.tank.level < totalCoffeeGrams:
            print(f"[{self.env.now}] {self.name}: No hay suficiente café para {group.name}, {group.table.name}, (pedido:{totalCoffeeGrams}g) (tank:{self.tank.level}/{self.tankCapacity}g)")
            df = pd.DataFrame({
                "time": [self.env.now],
                "robocat": [self.name],
                "table": [table.name],
                "group": [group.name],
                "group_size": [group.size],
                "event": [event.NOT_ENOUGH_COFFEE]
            })
            self.dfE = pd.concat( [ self.dfE , df ] )
        else:
            yield self.tank.get(totalCoffeeGrams)
            totalCoffeeTime = group.size * self.coffeeTime
            print( f"[{self.env.now}] {self.name}: Preparando {group.size} cafés en {totalCoffeeTime} minutos (newtank:{self.tank.level}/{self.tankCapacity}g)" )
            df = pd.DataFrame({
                "time": [self.env.now],
                "robocat": [self.name],
                "table": [table.name],
                "group": [group.name],
                "group_size": [group.size],
                "event": [event.MAKING_COFFEE]
            })
            self.dfE = pd.concat( [ self.dfE , df ] )

            yield self.env.timeout(totalCoffeeTime)
            group.wasAttended = True


In [265]:
class NyanCafe: # Cafetería
    def __init__(self, env, tableCapacity, spawnTime, groupSize, pMinDrinkingTime, pDrinkingTime, coffeeTime, gramsPerCoffee, tankCapacity, nRobocats, nTables, nGroups):
        self.env = env
        self.tableCapacity = tableCapacity
        self.spawnTime = spawnTime
        self.groupSize = groupSize
        self.pMinDrinkingTime = pMinDrinkingTime
        self.pDrinkingTime = pDrinkingTime
        self.coffeeTime = coffeeTime
        self.gramsPerCoffee = gramsPerCoffee
        self.tankCapacity = tankCapacity
        self.nGroups = nGroups
        self.groups = []

        self.tables = [
            Table(
                env, 
                f"Table{i}", 
                tableCapacity
            ) for i in range(nTables)
        ]
        self.robocats = [
            RoboCat(
                env, 
                f"RoboCat{i}", 
                tankCapacity, 
                coffeeTime
            ) for i in range(nRobocats)
        ]

        env.process(self.spawnGroups())

    def spawnGroups(self):
        for i in range( self.nGroups ):
            size = random.randint(1, self.groupSize)
            drinkingTime = random.randint(self.pMinDrinkingTime, self.pDrinkingTime)
            table = random.choice(self.tables)
            self.groups.append(
                Group(
                    self.env, 
                    f"Group{i}", 
                    size, 
                    drinkingTime, 
                    table, 
                    self.robocats
                )
            )
            yield self.env.timeout(random.randint(1, self.spawnTime))

## Simulación

In [266]:
random.seed(RANDOM_SEED)
env =  simpy.Environment()
nyanCafe = NyanCafe(
    env, 
    tableCapacity, 
    spawnTime, 
    groupSize, 
    pMinDrinkingTime, 
    pDrinkingTime, 
    coffeeTime, 
    gramsPerCoffee, 
    tankCapacity, 
    nRobocats=2, 
    nTables=3, 
    nGroups=5
)
env.run( until=480 ) # 8 horas de simulación


[0] Group0: Esta esperando la Table2
[0] Group0: Ha tomado la Table2
[0] Group0: Esta revisando Table2
[1] Group0: Encontró suficiente espacio en Table2 (gSize:1) (tSize:4)
[1] Group0: Esperando para pedir a RoboCat1
[1] RoboCat1: Escuchando a Group0 
[1] RoboCat1: Procesando pedido de Group0 en Table2 (pedido:5g) (tank:1000/1000g)
[1] RoboCat1: Preparando 1 cafés en 5 minutos (newtank:995/1000g)
[6] Group0: Recibió su café de RoboCat1 y se estará 10 minutos en la mesa
[16] Group1: Esta esperando la Table2
[16] Group0: Abandonó la mesa Table2
[16] Group1: Ha tomado la Table2
[16] Group1: Esta revisando Table2
[17] Group1: Encontró suficiente espacio en Table2 (gSize:2) (tSize:4)
[17] Group1: Esperando para pedir a RoboCat0
[17] RoboCat0: Escuchando a Group1 
[17] RoboCat0: Procesando pedido de Group1 en Table2 (pedido:10g) (tank:1000/1000g)
[17] RoboCat0: Preparando 2 cafés en 10 minutos (newtank:990/1000g)
[27] Group1: Recibió su café de RoboCat0 y se estará 14 minutos en la mesa
[41]

## Recolectar los Dataframes

In [267]:
df_events = pd.DataFrame()

In [268]:
df_robocats = pd.DataFrame()

for r in nyanCafe.robocats:
    df_robocats = pd.concat([ df_robocats , r.dfR ])
    df_events = pd.concat([ df_events , r.dfE ])

In [269]:
df_groups = pd.DataFrame()

for g in nyanCafe.groups:
    df_groups = pd.concat([ df_groups , g.dfG ])
    df_events = pd.concat([ df_events , g.dfE ])

In [270]:
df_tables = pd.DataFrame()

for t in nyanCafe.tables:
    df_tables = pd.concat([ df_tables , t.dfT ])
    df_events = pd.concat([ df_events , t.dfE ])

In [271]:
df_groups

,name,size,drinkingTime,table
0,Group0,1,10,Table2
0,Group1,2,14,Table2
0,Group2,1,28,Table1
0,Group3,1,16,Table0
0,Group4,2,30,Table2


In [272]:
df_robocats

,name,tankCapacity,coffeeTime
0,RoboCat0,1000,5
0,RoboCat1,1000,5


In [273]:
df_tables

,name,capacity
0,Table0,4
0,Table1,4
0,Table2,4


## Análisis de los dataframes de la simulación

### Cuál fue el primer y cuál fue el ultimó momento en que **inició a producir** café cada Robocat

In [274]:
df_events.loc[ 
    ( df_events["event"].isin([event.MAKING_COFFEE]) )
, :  ].groupby(
    "robocat"
).agg({
    "time": ["min", "max"]
})

time    
          min max
robocat          
RoboCat0   17  57
RoboCat1    1  90

### Total clientes atendidos y cuál es es tamaño promedio de los grupos en que llegan

In [277]:
total_customer_groups = df_events.loc[ df_events['event'] == event.DRINKING_COFFEE , : ].shape[0]

total_customers = df_events.loc[ df_events['event'] == event.DRINKING_COFFEE , 'group_size' ].sum()

average_customer_groups_size = df_events.loc[ df_events['event'] == event.DRINKING_COFFEE , 'group_size' ].mean()

print(f"Total de grupos de clientes: {total_customer_groups}")
print(f"Promedio de tamaño de grupos de clientes: {average_customer_groups_size}")
print(f"Total de clientes servidos: {total_customers}")



Total de grupos de clientes: 5
Promedio de tamaño de grupos de clientes: 1.4
Total de clientes servidos: 7


In [ ]:
attended = df_events.loc[ 
    df_events["event"].isin([ event.WAITING_IN_LINE , event.DRINKING_COFFEE ])
    , : 
].sort_values(["time"])
attended

,time,robocat,table,group,group_size,event
0,0,RoboCat1,Table2,Group0,1,WAITING_IN_LINE
0,16,RoboCat1,Table2,Group0,1,DRINKING_COFFEE
0,16,RoboCat0,Table2,Group1,2,WAITING_IN_LINE
0,41,RoboCat0,Table2,Group1,2,DRINKING_COFFEE
0,51,RoboCat0,Table1,Group2,1,WAITING_IN_LINE
0,53,RoboCat0,Table0,Group3,1,WAITING_IN_LINE
0,78,RoboCat0,Table0,Group3,1,DRINKING_COFFEE
0,85,RoboCat0,Table1,Group2,1,DRINKING_COFFEE
0,89,RoboCat1,Table2,Group4,2,WAITING_IN_LINE
0,130,RoboCat1,Table2,Group4,2,DRINKING_COFFEE
